(1) By defining the random variable of X as uniformly distribute between 0 and 1, the MC integration can be computed by generating n samples of X and computing the mean.

In [1555]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as st

def conf_int (mean, std, n):
       Z = st.t.ppf(0.95, n)
       return (mean - Z * (std/math.sqrt(n)), mean + Z * (std/math.sqrt(n)))

n_samples = 100

def monte_carlo_integration(n_samples):
    # Generate random samples uniformly distributed in [0, 1]
    samples = np.random.uniform(0, 1, n_samples)

    # Evaluate the function e^x at each sample point
    function_values = np.exp(samples)
    estimate = np.mean(function_values)
    variance =np.var(function_values)

    return estimate, variance

estimate, variance = monte_carlo_integration(n_samples)
print(f"Estimated integral: {estimate}, Variance: {variance}")
print(f"Reference: {np.exp(1) - 1}")

# 95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")

Estimated integral: 1.7144059665442293, Variance: 0.24756229917930767
Reference: 1.718281828459045
95% Confidence interval: [0.7391955643605629, 2.689616368727896]


We can observe that the confidence interval is quite large for this approximation (high variance).
The following methods are used to reduce applied to reduce this variance.

(2) For the estimation of the integral using antithetic variable, we compute the antithetic path for every rv of an uniformly distributed sample.

In [1566]:
def antithetic_integration(n_samples):
    # Generate random samples uniformly distributed in [0, 1]
    samples = np.random.uniform(0, 1, n_samples)

    # for every sample path, take its antithetic path and average the function values
    
    function_samples =( np.exp(samples) + np.exp(1 - samples) ) / 2

    estimate = np.mean(function_samples)
    variance =np.var(function_samples)

    return estimate, variance



estimate, variance = antithetic_integration(n_samples)
print(f"Estimated integral: {estimate}, Variance: {variance}")
print(f"Reference: {np.exp(1) - 1}")

#95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")


Estimated integral: 1.7244375159569298, Variance: 0.0037943188217164463
Reference: 1.718281828459045
95% Confidence interval: [1.603705352945138, 1.8451696789687217]


We can observe that the variance was significantly reduced when compared to the MC integration method.

(3) We compute the value of a control variable, c, by computing the covariance between a uniformly distributed sample Y and an exponentially distributed sample derived X from Y.

In [1568]:
def control_variates(n_samples):
    Y = np.random.uniform(0, 1, n_samples)
    X = np.exp(Y)

    cov1 = np.mean(Y*X) - np.mean(Y) * np.mean(X)
    var1 = np.var(Y)

    c = -cov1 / var1


    Z = X + c*(Y - np.mean(Y))
    
    estimate = np.mean(Z)
    variance =np.var(Z)

    return estimate, variance


estimate, variance = control_variates(n_samples)
print(f"Estimated integral: {estimate}, Variance: {variance}")
print(f"Reference: {np.exp(1) - 1}")

#95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")


Estimated integral: 1.710714718263442, Variance: 0.00363513696385033
Reference: 1.718281828459045
95% Confidence interval: [1.5925422080915435, 1.8288872284353404]


The variance appears to have reduced slightly compared to the antithetic variable integration method, but it's not significant.

(4) For the stratified sampling method, we sample in specific areas based on what we know about the sampling space.

In [1569]:
def stratified_sampling(n_samples):
    U = np.random.uniform(0, 1, n_samples)
    W = []

    #compute the stratified sample for each rv in u
    for u in U:
        sum = 0
        for j in range(10):
            sum += np.exp(j/10 + u/10 )
        #append the result to the final sample
        W.append(sum / 10)


    
    estimate = np.mean(W)
    variance =np.var(W)

    return estimate, variance

estimate, variance = stratified_sampling(n_samples)
print(f"Estimated integral: {estimate}, Variance: {variance}")
print(f"Reference: {np.exp(1) - 1}")

#95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")


Estimated integral: 1.7200744813492446, Variance: 0.0025698683075879723
Reference: 1.718281828459045
95% Confidence interval: [1.620714499010126, 1.819434463688363]


The variance was further reduced in comparison to the control variate integration method.

(5) We are able to reduce the variance of the estimate of the fraction of blocked customers by finding the control variate, obtained from the covariance between the sample containing the franction of blocked customers and the interarrival times.

In [1051]:
customers = 10000
k = 2.05
sim = 10



l=1

#---------------------------------------------------------------------------------------------------------------------
m = 10
s = 8.0
p_dist = np.random.exponential(1, size=customers)
exp_dist = np.random.exponential(8, customers)

    

def blocked_customers (inter_arrival_dist, service_times_dist, m, s, customers, sim):
    fract_blocked_counts = []
    blocked_customer_count = 0

    #set service time for all units to zero
    service_unit_time = np.zeros(m)

    for _ in range(sim):
        blocked_customer_count = 0
        #go through all customers
        for i in range(customers):
            #generate random arrival time for coming customer
            arrival_time = inter_arrival_dist[i]
            #subtract the arrival time from the service units times
            service_unit_time -= arrival_time

            #sort the service units with increasing time
            service_unit_time = np.sort(np.maximum(service_unit_time, 0.0))

            #check if there are service units 
            if (len([i for i in service_unit_time if i < 0.1]) == 0):
                blocked_customer_count += 1
            else:
                service_unit_time[0] = service_times_dist[i]
                    
        fract_blocked_counts.append(blocked_customer_count/customers)
    
    return fract_blocked_counts


customers = 10000
x_m = 8 * (2.05 - 1) / 2.05
l = 1

# Compute sample X, containing the fractions of blocked customers
X = blocked_customers (p_dist, exp_dist, 10, 8, customers, 10)

# Y is the sample for the interarrival times
Y = p_dist

new_estimates = []

# go through each fraction of blocked customers
for x in X:

    cov1 = np.mean(Y*x) - np.mean(Y) * x
    var1 = np.var(Y)

    #Compute the control variable
    c = -cov1 / var1

    #Compute the new estimate
    Z = X + c*(np.mean(Y) - 1/l)

    #Append new estimate to the final sample
    new_estimates.append(Z)

estimate = np.mean(new_estimates)
variance =np.var(new_estimates)

print(f"Estimate1: {estimate}, Variance1: {variance}")

#95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")


Estimate1: 0.11225999999999997, Variance1: 1.439999999999982e-08
95% Confidence interval: [0.11202479999999997, 0.11249519999999998]


The variance of the estimate is significantly reduced.

(6) We compute the sample containing the fractions of blocked customers from 20 simulations, by deriving the hyperexponentially distributed sample and the exponentially distributed sample from the same uniformly distributed sample of random varuiables, U. We then perform a 2-samples test statistic to verify if there is an effect between the means.

In [1547]:
customers = 10000
sim = 20
m = 10
s = 8
mean_tbc = 1
arrival_intensity = 1

u = np.random.uniform(0, 1, customers)

# Reprogramming the return value of the function and the interarrival distributions
def blocking_system_collect (inter_arrival_dist, service_times_dist, m, s, customers, sim):
  fract_blocked_counts = []
  blocked_customer_count = 0

  #set service time for all units to zero
  service_unit_time = np.zeros(m)

  for _ in range(sim):
    blocked_customer_count = 0
    #go through all customers
    for i in range(customers):
      #generate random arrival time for coming customer
      if (inter_arrival_dist == "hyper-exponential"):
        e = 0.8 * (-np.log(u)/0.8333) + 0.2 * (-np.log(u)/5)
        # EXERCISE 5, QUESTION 6 -> set arrival time to ith randomly generated hyperexponential dist. arrival time
        arrival_time =  e[i]
      elif (inter_arrival_dist == "erlang") :
        arrival_time = st.erlang.rvs(1, size=1, scale=mean_tbc)[0]
      else:
         # EXERCISE 5, QUESTION 6 -> set arrival time to ith randomly generated Poisson dist. arrival time
         arrival_time =  (-np.log(u)/l) [i]


      #subtract the arrival time from the service units times
      service_unit_time -= arrival_time

      #sort the service units with increasing time
      service_unit_time = np.sort(np.maximum(service_unit_time, 0.0))

      #check if there are service units 
      if (len([i for i in service_unit_time if i < 0.1]) == 0):
        blocked_customer_count += 1
      else:
        if (service_times_dist == "exponential"):
          service_unit_time[0] = np.random.exponential(8, 1)[0]
        elif (service_times_dist == "constant"):
          service_unit_time[0] = s
        elif (service_times_dist == "pareto_1.05"):
          x_m = s * (1.05 - 1) / 1.05
          service_unit_time[0] = (( np.random.pareto(1.05, size=1) + 1) * x_m ) [0]
        elif (service_times_dist == "pareto_2.05"):
          x_m = s * (2.05 - 1) / 2.05
          service_unit_time[0] = (( np.random.pareto(2.05, size=1) + 1) * x_m ) [0]
        elif (service_times_dist == "chi-squared"):
          service_unit_time[0] = np.random.chisquare(s, 1)[0]
        else:
           service_unit_time[0] = np.random.exponential(8, 1)[0]

        #service_unit_time[0] = service_times_dist[i]
                
    fract_blocked_counts.append(blocked_customer_count/customers)

  return fract_blocked_counts


In [1570]:
customers = 10000
l = 1

u = np.random.uniform(0, 1, customers)

#Poisson interarrival times based on inverse transform sampling
p_dist = -np.log(u)/l

# Hyperexponential interarrival times based on inverse transform sampling
hyperexp = 0.8 * (-np.log(u)/0.8333) + 0.2 * (-np.log(u)/5)

p_dist1 = np.random.exponential(1, size=customers)

blocked_pois = blocking_system_collect ("exponential", "exponential", m, s, customers, sim)
blocked_hyper = blocking_system_collect ("hyperexponential", "constant", m, s, customers, sim)


result = st.ttest_ind(a=blocked_pois, b=blocked_hyper, equal_var=False)

print(f"p-value = {result.pvalue}")
if (result.pvalue < 0.05):
  print("Reject null hypothesis")
else:
    print("Accept null hypothesis")


p-value = 0.28945047926492645
Accept null hypothesis


/var/folders/4l/6stdwbh56pvgwz8thwnlgtq80000gn/T/ipykernel_40213/2982088959.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  result = st.ttest_ind(a=blocked_pois, b=blocked_hyper, equal_var=False)


Since the p-value is greater than the significance level of 0.05, there is enough evidence to accept the null hypothesis, that the means between the two samples are equal.

(7) We use the Crude MC estimator to find the probability that a standard normal random variable Z is greater than a. Given a standard normal sample of random variables, we traverse through the array and count the number of variables that satisfy the condition and divide it by the size of the sample. We repeat this 100 times and compute the average and variance of the probabilities collected. 

In [1534]:

a = 2

n_samples = 100


def monte_carlo_estimator(n_samples, a, sigma):

    prob_collector = []
    for _ in range(100):
        samples = np.random.normal(0, 1, n_samples)
        # Check 
        greater_a = [x for x in samples if x > a]
        probs = len(greater_a) / n_samples
        prob_collector.append(probs)
        

    probability = np.mean(prob_collector)
    variance = np.var(prob_collector)

    return probability, variance





a_list = [2, 4]
sample_sizes = [10**3, 10**4, 10**5]
vars = [1, 2, 3]

# Print loop
for v in vars:
    for a in a_list:
        print(f"variance = {v}")
        print(f"a = {a}")
        for n in sample_sizes:
            estimate, variance = monte_carlo_estimator(n_samples, a, np.sqrt(v))
            print(f"  n = {n}: Estimate: {estimate}, Variance: {variance} ")




variance = 1
a = 2
  n = 1000: Estimate: 0.023899999999999998, Variance: 0.00021178999999999998 
  n = 10000: Estimate: 0.023899999999999998, Variance: 0.00027779 
  n = 100000: Estimate: 0.024199999999999996, Variance: 0.00027236 
variance = 1
a = 4
  n = 1000: Estimate: 0.0, Variance: 0.0 
  n = 10000: Estimate: 0.0, Variance: 0.0 
  n = 100000: Estimate: 0.0001, Variance: 9.9e-07 
variance = 2
a = 2
  n = 1000: Estimate: 0.021, Variance: 0.000235 
  n = 10000: Estimate: 0.023599999999999996, Variance: 0.00024704 
  n = 100000: Estimate: 0.023, Variance: 0.000269 
variance = 2
a = 4
  n = 1000: Estimate: 0.0, Variance: 0.0 
  n = 10000: Estimate: 0.0001, Variance: 9.899999999999998e-07 
  n = 100000: Estimate: 0.0001, Variance: 9.899999999999994e-07 
variance = 3
a = 2
  n = 1000: Estimate: 0.026099999999999995, Variance: 0.00022778999999999993 
  n = 10000: Estimate: 0.0264, Variance: 0.00026304 
  n = 100000: Estimate: 0.022899999999999997, Variance: 0.00018259 
variance = 3
a = 4


(7) For Importance Sampling, we define f(x) as the pdf of the standard normal distribution, the proposed distribution g(x) as the pdf of a normal distribution with mean a and standard deviation equal to the square root of a given variance. h(x) is defined as the indicator function that given a sample, replaces every variable by 1 if it is greater than a or by 0 if it isn't.

In [1536]:

def importance_sampling(a, n_samples, var1):

    # Compute sample average of I(Z > a)
    indicator = lambda l: 1 if l > a else 0

    # cdf of normal distribution (0, 1)
    def f(x) : return np.exp(-0.5*(x-0)**2)/ (1 * np.sqrt(2*np.pi))

    #cdf of normal distribution (a, var1)
    def g (x, a, o): return np.exp(-0.5*(x-a)**2)/ (o * np.sqrt(2*np.pi))

    def h(x) : return [indicator(i) for i in x]
    
    
    x = np.random.normal(0, 1, size=n_samples)

    # Proposal distribution
    g_dist = np.random.normal(a, math.sqrt(var1), size=n_samples)
    
    # Compute importance weights (pdf of normal function) h(x)/g(x)
    weights = f(g_dist) / g(g_dist, a, math.sqrt(var1))
    
   
    # p1 = [indicator(i) for i in x]
    # p1 = np.mean(p1)

    estimate = np.mean(weights * h(g_dist))
    variance = np.var(weights * h(g_dist))
    
    
    return estimate, variance

a_list = [2, 4]
sample_sizes = [10**3, 10**4, 10**5]
vars = [1, 2, 3]

# Print loop
for v in vars:
    for a in a_list:
        print(f"Variance = {v}")
        print(f"  a = {a}")
        for n in sample_sizes:
            estimate, variance = importance_sampling(a, n, v)
            print(f"    n = {n}: Estimate: {estimate}, Variance: {variance} ")



Variance = 1
  a = 2
    n = 1000: Estimate: 0.021999896129236543, Variance: 0.0011944171169602793 
    n = 10000: Estimate: 0.022981442467991737, Variance: 0.0012131586489093392 
    n = 100000: Estimate: 0.022895554328816556, Variance: 0.0012126650112229368 
Variance = 1
  a = 4
    n = 1000: Estimate: 3.224950666363084e-05, Variance: 4.7506189184903115e-09 
    n = 10000: Estimate: 3.070437976489888e-05, Variance: 4.333783473247412e-09 
    n = 100000: Estimate: 3.1615550776086144e-05, Variance: 4.5169079020710746e-09 
Variance = 2
  a = 2
    n = 1000: Estimate: 0.02398779898241075, Variance: 0.0018005557710144174 
    n = 10000: Estimate: 0.024886263265550407, Variance: 0.0019744440265507556 
    n = 100000: Estimate: 0.024293774489027656, Variance: 0.0018974125899436378 
Variance = 2
  a = 4
    n = 1000: Estimate: 2.4467733719319366e-05, Variance: 4.253424720398603e-09 
    n = 10000: Estimate: 3.0314460939562914e-05, Variance: 6.3812967099528585e-09 
    n = 100000: Estimate: 3

We can observe that the results are very similar between the two methods and that the Importance Sampling method is more precise in the estimates where the MC crude method computes an estimate of the probability that is equal to 0.

(8) Firstly the optimal value of lambda is obtained by calculating the variance of h(X)f(X)/g(X) and choosing the lambda for which the variance is the lowest. Finally we compute the mean of h(X)f(X)/g(X) using g(x) as the density function of X and the optimal value for lambda. 

In [1580]:
#actual distribution for the parameter we estimate
def h(x):
    return np.exp(x)

# pdf of proposed x distribution
def g(x, l):
    return l * np.exp(-l * x)

# pdf of right x distribution
#x is uniform and between 0 and 1, so pdf is equal to each generated number
def f(x):
    return  x

N = 10000


def indicator_function1(arr, a, b):
    return [1 if x >= a and x <= b else 0 for x in arr]

def optimal_lambda(N):
    variances = []

# check the optimal lambda value for the variance that is between 0.1 and 5 in steps of 0.1
    for l in np.arange(0.1, 5, 0.1):
        
        # distribution with pdf = g(x)
        x = np.random.exponential(1/l, size=N)

        # get random variables that are between 0 and 1
        p = indicator_function1(f(x), 0, 1)

        weights = p / g(x, l)
        est = weights * h(x)
        variance = np.var(est, ddof=1)
            
        variances.append(np.mean(variance))
    return np.argmin(variances)


optimal_lambda_val = [i for i in np.arange(0.1, 5, 0.1)] [optimal_lambda(N)]
print(f"Optimal lambda value: {optimal_lambda_val}")



def monte_carlo (n_samples, l):

    # distribution with pdf = g(x)
    samples = np.random.exponential(1/l, size=n_samples)

    # get random variables that are between 0 and 1
    p = indicator_function1(samples, 0, 1)

    weights = p / g(samples, l)
    est = weights * h(samples)
    return np.mean(est), np.var(est)

m, v = monte_carlo (N, optimal_lambda_val)
print(f"Estimate: {m}, Variance: {v}")
print(f"Reference: {np.exp(1) - 1}")




Optimal lambda value: 1.3000000000000003
Estimate: 1.721406740853014, Variance: 3.123076250684738
Reference: 1.718281828459045


We can observe that the estimate computed is close to the true value of the integral.

(9) It is meaningful to use the first moment distribution as a sampling distribution since it represents the expected value from some probability distribution. Therefore, since the first moment distribution is known, it can be used as a sampling distribution to derive an IS estimator. 
The purpose of choosing g(x) is mostly for cases when sampling from g(x) is easier than sampling from f(x). It is possible to chose a function g(x) such that the importance sampling would reduce the estimator, however it is not always guaranteed. Generally,  first moment distribution can be used as a sampling distribution to reduce the variance of an estimate.